In [6]:
import pandas as pd 
import numpy as np 
import bs4 
from selenium import webdriver
import requests 
import time
import re
import math
import seaborn as sns

In [7]:
titles = pd.read_csv('combined_titles_imdbinfo.csv').drop('Unnamed: 0', axis = 1)

In [8]:
test = titles.iloc[4, 3]

In [9]:
test

'tt3993886'

In [5]:
driver = webdriver.Chrome()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 81


In [ ]:
titles.imdbid = titles.imdbid.astype(str)

In [ ]:

titles

In [ ]:
titles.imdbid[0:5]

In [ ]:
imdb_dict = []
count = 0
for i, row in titles.iterrows():
    count += 1
    print(count)
    imdb_id = row.imdbid
    if imdb_id != 'nan':
        print(imdb_id)

        driver.get('https://imdb.com/title/' + imdb_id)
        time.sleep(.25)
        html = driver.page_source

        soup = bs4.BeautifulSoup(html)
        h_fours = soup.find_all('h4')
        for h in h_fours:
            if 'Genres' in str(h):
                genre = h.find_next_sibling().text.strip()
                break
            else: 
                genre = None
        for h in h_fours:
            if 'Country' in str(h):
                country = h.find_next_sibling().text.strip()
                break
            else:
                country = None
        for h in h_fours:
            if 'Language' in str(h):
                language = h.find_next_sibling().text.strip()
                break
            else:
                language = None
        for h in h_fours:
            if row['vtype'] == 'movie':
                if 'Runtime' in str(h):
                    runtime = h.find_next_sibling().text.strip()
            else:
                runtime = None
                
        rating_list = soup.find_all('span', {'class': 'rating'})
        print(rating_list)
        if rating_list:
            rating = rating_list[0].text
        else:
            rating = None

        
        t_blocks = soup.findAll('div', {'class': 'txt-block'})
        for t in t_blocks:
            if 'Release Date' in str(t):
                release_date = t.text.strip().split('\n')[0].split(':')[1].split(' (')[0].strip()
                break
            else:
                release_date = None
        for t in t_blocks:
            if 'Certificate' in str(t):
                certificate = t.find('span').text
                break
            else: certificate = None


        imdb_dict.append(
            dict(
                imdbid = imdb_id,
                genre = genre,
                rating = rating,
                country = country,
                language = language,
                runtime = runtime,
                release_date = release_date,
                certificate = certificate
            )
        )

In [ ]:
df_imdb = pd.DataFrame(imdb_dict)

In [ ]:
df_imdb.to_csv('imdb_details_7400.csv')

In [10]:
df_imdb_details = pd.read_csv('imdb_details_7400.csv')

In [11]:
df_imdb_details = df_imdb_details[df_imdb_details.imdbid != 'notfound']

In [12]:
df_combined = pd.merge(titles, df_imdb_details, how = 'left', on = ['imdbid']).drop_duplicates(['imdbid', 'title'])

In [72]:
df_combined.to_csv('all_details_26-5.csv')

In [ ]:
soup.find_all('div', {'class':'txt-block'})[1].find('span').text

In [7]:
titles.imdbid.value_counts()

notfound     558
tt1877368      4
tt2631186      4
tt4849438      3
tt5607970      3
            ... 
tt0244365      1
tt1019919      1
tt1580704      1
tt7055592      1
tt5235950      1
Name: imdbid, Length: 6784, dtype: int64

In [20]:
df_noid = titles[(titles.imdbid.isnull()) | (titles.imdbid == 'notfound')]

In [21]:
df_noid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1223 entries, 0 to 7693
Data columns (total 38 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   vtype           1223 non-null   object 
 1   img             1223 non-null   object 
 2   nfid            1223 non-null   int64  
 3   imdbid          558 non-null    object 
 4   title           1223 non-null   object 
 5   clist           1223 non-null   object 
 6   poster          0 non-null      object 
 7   imdbrating      9 non-null      float64
 8   top250tv        566 non-null    float64
 9   synopsis        1223 non-null   object 
 10  titledate       1223 non-null   object 
 11  avgrating       1223 non-null   float64
 12  year            1223 non-null   int64  
 13  runtime         1223 non-null   float64
 14  top250          567 non-null    float64
 15  id              1223 non-null   int64  
 16  IN              1039 non-null   object 
 17  UK              1050 non-null   o

In [22]:
titles.imdbid

0             NaN
1             NaN
2             NaN
3             NaN
4       tt3993886
          ...    
8060    tt0172156
8061    tt0257106
8062    tt1261945
8063    tt0366621
8064    tt0103060
Name: imdbid, Length: 8065, dtype: object

In [23]:
test = df_noid.iloc[0,]

In [24]:
test

vtype                                                         movie
img               https://occ-0-1168-299.1.nflxso.net/dnm/api/v6...
nfid                                                       81248053
imdbid                                                          NaN
title                                    My Long Awaited Love Story
clist                                                  "IN":"India"
poster                                                          NaN
imdbrating                                                      NaN
top250tv                                                        NaN
synopsis          A young romance game designer has no need for ...
titledate                                                2020-05-01
avgrating                                                         0
year                                                           2016
runtime                                                        5417
top250                                          

In [21]:
driver = webdriver.Chrome()

In [22]:
driver.get('https://www.imdb.com/')

In [23]:
driver.find_element_by_xpath('//*[@id="suggestion-search"]').send_keys(test['title'])


In [25]:
df_noid[['title', 'year']]

,title,year
0,My Long Awaited Love Story,2016
1,Perfect World,2019
2,Burning Love 2,2013
3,Burning Love,2012
9,Go! Go! Cory Carson: The Chrissy,2020
...,...,...
7132,Everybody Knows,2019
7133,Dr. Seuss&#39; The Grinch,2018
7134,Prosecuting Evil: The Extraordinary World of B...,2019
7135,Herrens veje,2018


In [29]:
df_id = titles[(titles.imdbid.notnull()) & (titles.imdbid != 'notfound')]

In [30]:
df_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6842 entries, 4 to 8064
Data columns (total 38 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   vtype           6842 non-null   object 
 1   img             6842 non-null   object 
 2   nfid            6842 non-null   int64  
 3   imdbid          6842 non-null   object 
 4   title           6842 non-null   object 
 5   clist           6842 non-null   object 
 6   poster          5998 non-null   object 
 7   imdbrating      6770 non-null   float64
 8   top250tv        6668 non-null   float64
 9   synopsis        6841 non-null   object 
 10  titledate       6842 non-null   object 
 11  avgrating       6842 non-null   float64
 12  year            6842 non-null   int64  
 13  runtime         6840 non-null   float64
 14  top250          6825 non-null   float64
 15  id              6842 non-null   int64  
 16  IN              4667 non-null   object 
 17  UK              4979 non-null   o

In [31]:
id_in = len(df_id[(df_id.IN == True) & (df_id.US.isnull()) & (df_id.UK.isnull())]) / len(df_id)

In [32]:
id_us = len(df_id[(df_id.US == True) & (df_id.IN.isnull()) & (df_id.UK.isnull())]) / len(df_id)

In [33]:
id_uk = len(df_id[(df_id.UK == True) & (df_id.IN.isnull()) & (df_id.US.isnull())]) / len(df_id)

In [34]:
id_in_us = len(df_id[(df_id.IN == True) & (df_id.US == True) & (df_id.UK.isnull())]) / len(df_id)

In [35]:
id_in_uk = len(df_id[(df_id.IN == True) & (df_id.UK == True) & (df_id.US.isnull())]) / len(df_id)

In [36]:
id_us_uk = len(df_id[(df_id.US == True) & (df_id.UK == True) & (df_id.IN.isnull())]) / len(df_id)

In [37]:
id_all = len(df_id[(df_id.US.notnull()) & (df_id.UK.notnull()) & (df_id.IN.notnull())]) / len(df_id)

In [38]:
#test for the no ID entries
noid_all = len(df_noid[(df_noid.IN.notnull()) & (df_noid.US.notnull()) & (df_noid.UK.notnull())]) / len(df_noid)

In [39]:
noid_us = len(df_noid[(df_noid.US == True) & (df_noid.IN.isnull()) & (df_noid.UK.isnull())]) / len(df_noid)

In [40]:
noid_uk = len(df_noid[(df_noid.UK == True) & (df_noid.IN.isnull()) & (df_noid.US.isnull())]) / len(df_noid)

In [41]:
noid_in = len(df_noid[(df_noid.IN == True) & (df_noid.US.isnull()) & (df_noid.UK.isnull())]) / len(df_noid)

In [42]:
noid_in_us = len(df_noid[(df_noid.IN == True) & (df_noid.US == True) & (df_noid.UK.isnull())]) / len(df_noid)

In [43]:
noid_us_uk = len(df_noid[(df_noid.US == True) & (df_noid.UK == True) & (df_noid.IN.isnull())]) / len(df_noid)

In [44]:
noid_in_uk = len(df_noid[(df_noid.IN == True) & (df_noid.UK == True) & (df_noid.US.isnull())]) / len(df_id)

In [60]:
titles['dbid_test'] = np.where((titles.imdbid.isnull()) | (titles.imdbid == 'notfound'), 0, 1)

In [61]:
titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8065 entries, 0 to 8064
Data columns (total 39 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   vtype           8065 non-null   object        
 1   img             8065 non-null   object        
 2   nfid            8065 non-null   int64         
 3   imdbid          7400 non-null   object        
 4   title           8065 non-null   object        
 5   clist           8065 non-null   object        
 6   poster          5998 non-null   object        
 7   imdbrating      6779 non-null   float64       
 8   top250tv        7234 non-null   float64       
 9   synopsis        8064 non-null   object        
 10  titledate       8065 non-null   datetime64[ns]
 11  avgrating       8065 non-null   float64       
 12  year            8065 non-null   int64         
 13  runtime         8063 non-null   float64       
 14  top250          7392 non-null   float64       
 15  id  

In [62]:
titles.dbid_test.value_counts()

1    6842
0    1223
Name: dbid_test, dtype: int64

In [63]:
titles.titledate = pd.to_datetime(titles.titledate)

In [81]:
titles['days_back'] = pd.to_numeric(pd.to_datetime('today') - titles.titledate)

In [84]:
titles[['days_back', 'dbid_test']].corr()

,days_back,dbid_test
days_back,1.00000,0.30245
dbid_test,0.30245,1.00000


In [86]:
df_noid.IN.value_counts()

True    1039
Name: IN, dtype: int64

In [87]:
df_noid.US.value_counts()

True    1079
Name: US, dtype: int64

In [88]:
df_noid.UK.value_counts()

True    1050
Name: UK, dtype: int64

In [14]:
df_combined.columns

Index(['vtype', 'img', 'nfid', 'imdbid', 'title', 'clist', 'poster',
       'imdbrating', 'top250tv', 'synopsis', 'titledate', 'avgrating', 'year',
       'runtime_x', 'top250', 'id', 'IN', 'UK', 'US', 'matlevel',
       'imdblanguage', 'imdbposter', 'lgimg', 'imdbawards', 'curdate',
       'nfdate', 'imdbruntime', 'imdblocalimage', 'matlabel', 'netflixruntime',
       'imdbcountry', 'predrating', 'imdbmetascore', 'imdbrated', 'imdbgenre',
       'netflixid', 'imdbplot', 'imdbvotes', 'Unnamed: 0', 'genre', 'rating',
       'country', 'language', 'runtime_y', 'release_date', 'certificate'],
      dtype='object')

In [51]:
df_tests = df_combined[['IN', 'UK', 'US', 'rating', 'country']]

In [52]:
df_tests.dropna(subset = ['rating'], inplace = True)

In [53]:
df_tests['rating'] = df_tests['rating'].apply(lambda x: x.split('/', 1)[0])

In [54]:
df_tests['rating'] = pd.to_numeric(df_tests.rating)

In [55]:
df_tests.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6696 entries, 4 to 8198
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   IN       4554 non-null   object 
 1   UK       4862 non-null   object 
 2   US       4712 non-null   object 
 3   rating   6696 non-null   float64
 4   country  6599 non-null   object 
dtypes: float64(1), object(4)
memory usage: 313.9+ KB


In [106]:
df_tests.dropna(subset = ['country'], inplace = True)

In [107]:
import statsmodels.api as sm
import pylab
import scipy
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

In [120]:
def oh_mann(data, cat_country, origin_one, origin_two):
    df_catalog = data[df_tests[cat_country] == True]
    df_sample_one = df_catalog[df_catalog.country.str.contains('|'.join(origin_one))]
    df_sample_two = df_catalog[~df_catalog.country.str.contains('|'.join(origin_two))]
    print(df_sample_two.rating.mean())
    print(df_sample_one.rating.mean())
    return scipy.stats.mannwhitneyu(df_sample_one.rating, df_sample_two.rating)

In [123]:
oh_mann(df_tests, 'US', ['USA'], ['USA'])

6.5517441860465295
6.479166666666672


MannwhitneyuResult(statistic=2462212.5, pvalue=0.005361585662818078)

MannwhitneyuResult(statistic=558092.5, pvalue=0.007884630023865248)